# Initial data analysis of Precipitation data
This notebook contains EDA of Precipitation data in order to gain information for the ETL process

The main point of the analysis is to find out how to extract the precipitation for city of Chicago from the raw data.

In [2]:
import os
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local') \
    .appName('precipitation') \
    .getOrCreate()

## Weather station data
Read the weather station data and have a peek at the schema and initial rows.

The file was sapce delimited, with varying amount of whitespace between columns. We'll handle this by first reading the file to an RDD, parse the columns and convert it to a DataFrame.

In [54]:
import re
data_folder = '/Users/tomra/Projects/data-engineering/udacity-data-engineer-nanodegree/06-capstone-project/data'

lines = spark.sparkContext.textFile(
    os.path.join(data_folder, 'ghcnd', 'ghcnd-stations.txt')
)
split_lines = lines.map(lambda line: re.split('\\s+', line))
final_lines = split_lines.map(lambda line: (line[0], line[1], line[2], line[3], ' '.join(line[4:-1])))
df_stations = final_lines.toDF() \
    .selectExpr(
        "cast(_1 as string) as id",
        "cast(_2 as float) as lat",
        "cast(_3 as float) as lon",
        "cast(_4 as float) as val",
        "cast(_5 as string) as station")

In [56]:
df_stations.printSchema()
df_stations.show(5)

root
 |-- id: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- lon: float (nullable = true)
 |-- val: float (nullable = true)
 |-- station: string (nullable = true)

+-----------+-------+--------+----+--------------------+
|         id|    lat|     lon| val|             station|
+-----------+-------+--------+----+--------------------+
|ACW00011604|17.1167|-61.7833|10.1|ST JOHNS COOLIDGE...|
|ACW00011647|17.1333|-61.7833|19.2|            ST JOHNS|
|AE000041196| 25.333|  55.517|34.0|SHARJAH INTER. AI...|
|AEM00041194| 25.255|  55.364|10.4|          DUBAI INTL|
|AEM00041217| 24.433|  54.651|26.8|      ABU DHABI INTL|
+-----------+-------+--------+----+--------------------+
only showing top 5 rows



## Precipitation data
Next we'll read the actual precipitation data file and:
* check amount of data
* and have a peek at the top rows

In [123]:
from pyspark.sql.functions import to_timestamp

df = spark.read \
    .format('csv') \
    .load(os.path.join(data_folder, 'ghcnd', '2016.csv')) \
    .selectExpr(
        "cast(_c0 as string) as id",
        "cast(_c1 as string) as date",
        "cast(_c2 as string) as element",
        "cast(_c3 as float) as value",
        "cast(_c4 as string) as m",
        "cast(_c5 as string) as q",
        "cast(_c6 as string) as s",
        "cast(_c7 as string) as time")
df = df.withColumn('date', to_timestamp(df.date, 'yyyyMMdd'))

In [124]:
df.count()

35326976

In [89]:
df.limit(5).toPandas()

,id,date,element,value,m,q,s,time
0,US1FLSL0019,2016-01-01,PRCP,3.0,None,None,N,None
1,NOE00133566,2016-01-01,TMAX,95.0,None,None,E,None
2,NOE00133566,2016-01-01,TMIN,23.0,None,None,E,None
3,NOE00133566,2016-01-01,PRCP,10.0,None,None,E,None
4,USC00141761,2016-01-01,TMAX,22.0,None,None,7,0700


## Merge data
The data file is to little use for us unless we can find a way to udnerstand where the weather station id's reside geographically.

We'll make a left join of the station data on the precipication data to have human readable column for the location of each row.

In [181]:
merged_df = df.join(df_stations, df.id == df_stations.id, 'left') \
    .select(df.id,
            df.date,
            df.element,
            df.value,
            df.time,
            df_stations.station)

Let's see if we can find some data from Chicago

In [182]:
filtered_df = merged_df.filter(merged_df.station.contains('CHICAGO'))

In [183]:
filtered_df.filter(filtered_df.element == "PRCP").groupBy('station').count().toPandas()

,station,count
0,IL CHICAGO 4.8 NNW,3
1,IL CHICAGO MIDWAY AP 3SW,366
2,IL WEST CHICAGO 2.7 N,85
3,IL CHICAGO BOTANIC GARDEN,365
4,IL CHICAGO PALWAUKEE AP,363
5,IN (KB9UUO)NEW CHICAGO 0.6 SSE,2
6,IL CHICAGO AURORA MUNI AP,360
7,IL CHICAGO 5.5 ESE,360
8,IL CHICAGO 6.4 NNE,34
9,IL CHICAGO WAUKEGAN RGNL AP,363


The Chicago Midway Airport contains precipitation for every day (2016 was a leap year) so we'll settle on one of them.

As a result we'll check the station ID so we can filter the data in ETL process more efficiently. Since we're only interested in the weather for one location there will be little benefit in repeating this join process in the ETL.

In [179]:
df_stations.filter(df_stations.station == 'IL CHICAGO MIDWAY AP 3SW').show()

+-----------+-------+--------+-----+--------------------+
|         id|    lat|     lon|  val|             station|
+-----------+-------+--------+-----+--------------------+
|USC00111577|41.7372|-87.7775|189.0|IL CHICAGO MIDWAY...|
+-----------+-------+--------+-----+--------------------+



Final filtering and creation of DataFrame for checking basic statistical information about the precipitation data.

In [184]:
precip_df = merged_df \
    .filter(merged_df.element == 'PRCP') \
    .filter(merged_df.id == 'USC00111577')

In [185]:
precip_df.count()

366

In [187]:
precip_df.describe().toPandas()

,summary,id,element,value,time,station
0,count,366,366,366,366,366
1,mean,None,None,29.1448087431694,2400.0,None
2,stddev,None,None,73.57138598660973,0.0,None
3,min,USC00111577,PRCP,0.0,2400,IL CHICAGO MIDWAY AP 3SW
4,max,USC00111577,PRCP,508.0,2400,IL CHICAGO MIDWAY AP 3SW


## Summary
We've successfully joined the station data with measurement records, identified weather stations in Chicago, chosen one, and as a result extracted daily precipitation values that can be used in the weather facts table.

We'll filter the raw data in ETL process like done above:
* Only include station id `USC00111577` that corresponds to Chicago Midway Station weather station
* Only incldue element value `PRCP` that corresponds to precipitation data